# ResNet 직접 구현하기
---
**"ResNet을 직접 구현하고 Plain Network와 비교하여 그 성능을 확인하는 프로젝트"**
<br>
#### 😏 목차
1. ResNet 구현
2. Plain Network 구현
3. 성능 비교
4. 회고

#### 모듈

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

---
## 1. ResNet 구현

### 1-1 ResNet 블록 구성

### 1-2 ResNet-34

### 1-3 ResNet-50

---
## 2. Plain Network 구현

### 2-1 Plain Network 블록 구성

### 2-2 Plain Network-34

### 2-3 Plain Network-50

---
## 3. 성능 비교

### 3-1 데이터 준비
- tensorflow-datasets의 cats_vs_dogs 데이터

### 3-2 ResNet-34 vs Plain Network-34 비교

#### ResNet-34

#### Plain Network-34

#### 시각화

---
## 4. 회고